In [9]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import pyttsx3

In [10]:
engine  = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[1].id) # 0 - male

In [11]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
landmarks = ['class']  # Class is Y - target value
for val in range(1,33+1):
    landmarks+=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

Deadlift Model

In [5]:
with open('deadlift.pkl','rb') as f:
    deadlift_model = pickle.load(f)

Lean Model

In [6]:
with open('lean.pkl','rb') as f:
    lean_model = pickle.load(f)

Distance Model

In [7]:
with open('distance.pkl','rb') as f:
    distance_model = pickle.load(f)

Final Code

In [33]:
cap = cv2.VideoCapture(0)
current_Stage = ''
current_lean_Status = ''
current_distance_Status = ''
counter = 0 
speak_set = 0
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recolor it back
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Draw Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        try:
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            bodylang_prob = deadlift_model.predict_proba(X)[0]
            bodylang_class = deadlift_model.predict(X)[0]
            lean_prob = lean_model.predict_proba(X)[0]
            lean_class = lean_model.predict(X)[0]
            distance_prob = distance_model.predict_proba(X)[0]
            distance_class = distance_model.predict(X)[0]
            if(lean_class==1.0 and lean_prob[lean_prob.argmax()]>0.5):
                # if speak_set==0:
                #     speak("Your Posture is perfect. Start the exercise.")
                #     speak_set=1
                if bodylang_class==0.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                    current_Stage = "down"
                elif bodylang_class==1.0 and current_Stage=="down" and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                    current_Stage = "up"
                    counter+=1
            # Setting Distance Status
            if distance_class==0.0 and distance_prob[distance_prob.argmax()] > 0.5:
                current_distance_Status = "Narrow"
            elif distance_class==1.0 and distance_prob[distance_prob.argmax()] > 0.5:
                current_distance_Status = "Correct"
            elif distance_class==2.0 and distance_prob[distance_prob.argmax()] > 0.5:
                current_distance_Status = "Wide"
            # Setting Lean Status
            if lean_class==0.0 and lean_prob[lean_prob.argmax()] > 0.5:
                current_lean_Status = "Left"
            elif lean_class==1.0 and lean_prob[lean_prob.argmax()] > 0.5:
                current_lean_Status = "Centre"
            elif lean_class==2.0 and lean_prob[lean_prob.argmax()] > 0.5:
                current_lean_Status = "Right"
            
            # Get status box
            cv2.rectangle(image,(0,0),(550,60),(245,117,16),-1)

            # Display Rep
            cv2.putText(image,'Status',
                        (95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_Stage,
                            (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Probability
            cv2.putText(image,'Prob',
                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(bodylang_prob[np.argmax(bodylang_prob)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Count
            cv2.putText(image,'Count',
                        (180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),
                        (200,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Lean
            cv2.putText(image,'Posture',
                        (280,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_lean_Status,
                            (280,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Distance
            cv2.putText(image,'Distance',
                        (400,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,"Correct",
                            (400,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Deadlifts',image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        except Exception as e:
            print(e)
cap.release()
cv2.destroyAllWindows()
            
            

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute